In [9]:
from q_value_calc_crosslinks import calcQ
from idXML2df_all_hits import readAndProcessIdXMLAllHits
from functions import get_target_id, read_top_down_data, get_datasets, rerank
import re
import os

In [10]:
i = 5
dataset = get_datasets()

In [11]:
directory = f"../data/{dataset[i]['type']}/"
input_files = [directory + f"{dataset[i]['file']}{dataset[i]['file_ending']}"]
if dataset[i]['comparison'] != None: 
      input_files.append(directory + f"{dataset[i]['file']}_{dataset[i]['comparison']}{dataset[i]['file_ending']}")
print(input_files)

['../data/top_down_data/outprsm4_multihits.tsv']


In [12]:
for input_file in input_files:
    if dataset[i]['type'] == 'crosslink_data':
        df = readAndProcessIdXMLAllHits(input_file)
        classColName = 'NuXL:isXL'
    elif dataset[i]['type'] == 'top_down_data':
        df = read_top_down_data(input_file)
        df['ScanNr'] = df['Scan']
        display(df)
        print(df.columns)
        df = rerank(df, 'ScanNr', 'Score', 'PSMId')
        classColName = 'ModCount'
    file_ending = dataset[i]['file_ending']
    directory += f"{dataset[i]['name']}/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    df.to_pickle(directory + f"{dataset[i]['file']}" + ".pkl")
    # filter dataframes
    filter_col = 'PSMId'
    filter_val = 1
    df = df.loc[df[filter_col] == filter_val,:]
    # compute q-values
    df.sort_values(['Score', 'Label'], ascending=[False, True], inplace=True)
    q_vals = calcQ(df, classColName = classColName)
    # compute target IDs
    target_id = get_target_id(q_vals, isXLColName=dataset[i]['group'])
    target_id.to_pickle(directory + f"{dataset[i]['file']}" + "_filtered.pkl")

,ProteoformIndex,Scan,RetentionTime,NumMass,ProteinAccession,ProteinDescription,PrecursorMass,ProteoformMass,DatabaseSequence,ProteinSequence,...,ModID,ModAccession,ModStart,ModEnd,PrecursorQscore,Score,PrSMLevelQvalue,ProteoformLevelQvalue,Label,ScanNr
0,0,968,453.432560,9,DECOY_sp|P77260|YDFI_ECOLI,Uncharacterized oxidoreductase YdfI OS=Escheri...,5192.565498,5192.565498,GEDYKCRSQLAGGQVAATVVQSQAVSTNLMVPIQPTDFLGHQVLPK...,KDILTNAAWTSTVHSHFYGVHDRAKDRLASLPAIYAHVRKTSA,...,"Biotin:Thermo-21328,","1841,",446,485,0.91032,4.01,0.287037,0.359726,0.0,968
1,1,968,453.432560,9,sp|P77754|SPY_ECOLI,Periplasmic chaperone Spy OS=Escherichia coli ...,5192.565498,5192.565498,MRKLTALFVASTLALGAANLAHAADTTTAAPADAKPMMHHKGKFGP...,KDLNLTDAQKQQIREIMKGQRDQMKRPPLEERRAMHDIIAS,...,"PhosphoUridine,","417,",56,93,0.91032,4.01,0.287037,0.359726,1.0,968
2,2,968,453.432560,9,DECOY_sp|P33346|YEHI_ECOLI,Uncharacterized protein YehI OS=Escherichia co...,5192.565498,5192.565498,VTVELILMIEAEWKAARVPEGKPAELGLEELLRAQLWYLGALQLAP...,KDLTVKRHASLTRWIAGGTEEGSKPGELRPALPEDCLTRGSAAKYL,...,"SPITC,IDEnT,","261,762,",588,630,0.91032,4.01,0.287037,0.359726,0.0,968
3,3,968,453.432560,9,sp|P23909|MUTS_ECOLI,DNA mismatch repair protein MutS OS=Escherichi...,5192.565498,5192.565498,MSAIENFDAHTPMMQQYLRLKAQHPEILLFYRMGDFYELFYDDAKR...,KVEIGPIDRIFTRVGAADDLASGRSTFMVEMTETANILHNATEYSL,...,"Glycerophospho,Delta:H(6)C(7)O(4),UgiJoulliePr...","419,1929,1282,721,",647,689,0.91032,8.01,0.268200,0.343805,1.0,968
4,4,968,453.432560,9,DECOY_sp|P23538|PPSA_ECOLI,Phosphoenolpyruvate synthase OS=Escherichia co...,5192.565498,5192.565498,LSMVDDQVIKNAGITFFPEKVPPRLNVFGMANGALSAKTQLKDFVT...,KDIWDIQPRLAGSKVTIEIIGGTTFPETSRESDYGPQFVIIRGATY,...,"dichlorination,","937,",581,623,0.91032,4.01,0.287037,0.359726,0.0,968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57016,57016,14646,5399.224109,36,sp|P52108|RSTA_ECOLI,Transcriptional regulatory protein RstA OS=Esc...,-1.000000,-1.000000,MNTIVFVEDDAEVGSLIAAYLAKHDMQVTVEPRGDQAEETILRENP...,MNTIVFVEDDAEVGSLIAAYLAKHDMQVTVEPRGDQAEETILRENP...,...,"Delta:H(6)C(6)O(1),","205,",120,133,0.00000,16.01,0.465485,0.496234,1.0,14646
57017,57017,14646,5399.224109,36,sp|P0AGC3|SLT_ECOLI,Soluble lytic murein transglycosylase OS=Esche...,-1.000000,-1.000000,MEKAKQVTWRLLAAGVCLLTVSSVARADSLDEQRSRYAQIKQAWDN...,ENARLMIPSLAQAQQLNEDQIQELRDIVAWRLMGNDVTDEQAKWRD...,...,NaN,NaN,258,283,0.00000,8.01,0.490398,0.496821,1.0,14646
57018,57018,14646,5399.224109,36,sp|P0ACD8|MBHL_ECOLI,Hydrogenase-1 large chain OS=Escherichia coli ...,-1.000000,-1.000000,MSTQYETQGYTINNAGRRLVVDPITRIEGHMRCEVNINDQNVITNA...,MSTQYETQGYTINNAGRRLVVDPITRIEGHMRCEVNINDQNVITNA...,...,NaN,NaN,328,353,0.00000,8.01,0.490398,0.496821,1.0,14646
57019,57019,14646,5399.224109,36,DECOY_sp|P0DP64|YBEM_ECOLI,Putative protein YbeM OS=Escherichia coli (str...,-1.000000,-1.000000,QATCGEHLHAIEVNGKRAEGWAQLEFEVELTDDLLVEVVPPMLAYG...,QATCGEHLHAIEVNGKRAEGWAQLEFEVELTDDLLVEVVPPMLAYG...,...,NaN,NaN,59,86,0.00000,1.01,0.495504,0.504719,0.0,14646


Index(['ProteoformIndex', 'Scan', 'RetentionTime', 'NumMass',
       'ProteinAccession', 'ProteinDescription', 'PrecursorMass',
       'ProteoformMass', 'DatabaseSequence', 'ProteinSequence', 'Proforma',
       'MatchingFragments', 'Coverage(%)', 'StartPosition', 'EndPosition',
       'TagCount', 'TagIndices', 'ModCount', 'ModMass', 'ModID',
       'ModAccession', 'ModStart', 'ModEnd', 'PrecursorQscore', 'Score',
       'PrSMLevelQvalue', 'ProteoformLevelQvalue', 'Label', 'ScanNr'],
      dtype='object')


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


In [13]:
import pandas as pd
import numpy as np
input_file_original = f"../data/{dataset[i]['type']}/{dataset[i]['file']}_filtered.pkl"
df = pd.read_pickle(input_file_original)
df.loc[df['class-specific_q-val'] > 0,['ModCount', 'Label','FP', 'TagCount', 'Count', 'class-specific_FDR', 'class-specific_q-val', 'Score', 'PrSMLevelQvalue', 'ProteoformLevelQvalue' ]].head(50)
# tag count /100 + score

KeyError: "['FP', 'Count', 'class-specific_FDR'] not in index"